# Reanding Exif data from ./images

In [16]:
import os
from PIL import Image

dir = "images"
themes = os.listdir(dir)
data = []

for theme in themes: 
    if theme in {".DS_Store", "ExifData.json","ExifData2.json","ExifDatatest.json"}:
        pass   
    else:
        print(theme)
        dir_test = os.path.join(dir, theme)
        for image in os.listdir(dir_test):
            #print(image)
            if image.endswith(".jpg") :
                #img_path =  dir + theme + '/'+ image
                img_path = os.path.join(dir, theme, image)
                # print(img_path)
                with Image.open(img_path) as img:
                    # print(img)
                    width, height = img.size
                    format = img.format
                    try:
                        orientation = img._getexif()[274]
                    except:
                        orientation = "no exif"
                    #print(f'{width=}x{height=} {format=} {orientation=}')
                    data.append({"theme": theme, "image": image, "width": width, "height": height, "format": format, "orientation": orientation})
                    
print(data[0:5])

Exoplanet
Pokemon
Moto
Voitures
[{'theme': 'Exoplanet', 'image': '55_Cancri_b.jpg', 'width': 1024, 'height': 768, 'format': 'PNG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'Gliese_876_e.jpg', 'width': 1587, 'height': 919, 'format': 'PNG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'XO-3b.jpg', 'width': 800, 'height': 500, 'format': 'JPEG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'OGLE-2005-BLG-169Lb.jpg', 'width': 1271, 'height': 746, 'format': 'JPEG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'Kepler-242b.jpg', 'width': 1980, 'height': 1200, 'format': 'JPEG', 'orientation': 'no exif'}]


## Get proiminant colors from images

In [2]:
from PIL import Image
import warnings
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import MiniBatchKMeans

def getKmeans(img):
    # init
    img = Image.open(img) # open image
    imgfile = img.resize((int(img.size[0]/5), int(img.size[1]/5))) # resize
    numarray = numpy.array(imgfile.getdata(), numpy.uint8) # convert image to numpy array
    cluster_count = 5 # number of clusters
    clusters = MiniBatchKMeans(n_clusters=cluster_count, n_init="auto") # init kmeans
    clusters.fit(numarray) # fit kmeans
    npbins = numpy.arange(0, cluster_count + 1) # init bins
    histogram = numpy.histogram(clusters.labels_, bins=npbins) # get histogram
    #labels = numpy.unique(clusters.labels_) # get labels

    # getting colors
    color = []
    for i in range(cluster_count):
        color.append(
            "#%02x%02x%02x" % (math.ceil(clusters.cluster_centers_[i][0]),math.ceil(clusters.cluster_centers_[i][1]),math.ceil(clusters.cluster_centers_[i][2]),)
        )
    #barlist = plot.bar(labels, sorted(histogram[0], reverse=True), color=color)
    #plot.show()
    return (color)

getKmeans("./images/Exoplanet/2M1207_b.jpg")

['#0c0d0f', '#a39284', '#4d4035', '#806956', '#1f2124']

## Getting histogram from images

In [18]:
from PIL import Image
import matplotlib.pyplot as plot
import numpy as np

def getDiagram(img):
    imgfile = Image.open(img)
    histogram = imgfile.histogram()
    red = histogram[0:255]
    green = histogram[256:511]
    blue = histogram[512:767]
    # x = range(255)
    # y = []
    # for i in x:
    #     y.append((red[i], green[i], blue[i]))
    # figure,axes = plot.subplots()
    # axes.set_prop_cycle("color", ["red", "green", "blue"])
    # plot.plot(x, y)
    # plot.show()
    mapping = {"red": np.average(range(0,255),weights=red), "green": np.average(range(0,255),weights=green), "blue": np.average(range(0,255),weights=blue)}
    return mapping

getDiagram("./images/Exoplanet/2M1207_b.jpg")

{'red': 44.77778330018979,
 'green': 41.671026930561624,
 'blue': 39.357752621028695}

## Adding diagram and Kmeans to exif data

In [3]:
for obj in data:
    dir = os.path.join("images", obj["theme"], obj["image"])
    print(dir)
    colors = getKmeans(dir)
    for i, c in enumerate(colors):
        obj["color_{i}"] = c
    RGB = getDiagram(dir)
    for key, value in RGB.items():
        obj[key] = value

NameError: name 'data' is not defined

## Writing exif data into json file : ./images/ExifData.json

In [20]:
import json
from numpyencoder import NumpyEncoder

with open('./images/ExifDatatest.json', 'w+') as f:
    json.dump(data, f, indent=4, cls=NumpyEncoder)